In [6]:
import scipy
import pandas
import numpy
import scanpy
from pathlib import Path

In [7]:
analysis_dir = Path('ENCSR874BOF_e10_5_limb')

In [8]:
cellr_common = scanpy.read_h5ad(analysis_dir / 'cellranger_filtered.sparse.h5ad')
cellr_common

AnnData object with n_obs × n_vars = 6287 × 81881 
    var: 'gene_symbols', 'feature_types'

In [9]:
solo_common = scanpy.read_h5ad(analysis_dir / 'solo_filtered.sparse.h5ad')
solo_common

AnnData object with n_obs × n_vars = 6287 × 81881 
    var: 'gene_symbols'

In [21]:
solo_common.X.todense().shape

(6287, 81881)

In [10]:
dense_10x = {
    'cellranger': cellr_common.to_df(),
    'solo': solo_common.to_df(),
    #'alevin': alevin_common.to_df(),
    #'kallisto': kallisto_common.to_df(),
    #'kallisto EM': numpy.array(kallisto_em_common.X.todense()),
}

In [11]:
def correlate_matricies_pearson(left, right):
    assert left.shape == right.shape, "Shapes must be the same"
    cs_cors=[]
    for i in range(left.shape[1]):
        cs_cors.append(numpy.corrcoef(left[:,i], right[:,i])[0,1])
    cs_cors = numpy.array(cs_cors)[~numpy.isnan(cs_cors)]
    return cs_cors


In [16]:
def compute_correlations(dense):
    programs = list(dense.keys())
    cell_correlations = {}

    for name_x in programs:
        for name_y in programs[programs.index(name_x):]:
            assert dense[name_x].shape == dense[name_y].shape
            for cell_id in dense[name_x].columns:
                #cs_cors = replicate_scores(dense[name_x][cell_id], dense[name_y][cell_id], name_x, name_y)
                cs_cors = correlate_matricies_pearson(dense[name_x], dense[name_y])
                cell_correlations.setdefault(name_x, {}).setdefault(name_y, {})[cell_id] = cs_cors
    return cell_correlations

In [19]:
dense_10x['cellranger'].columns

Index(['896', '897', '1275', '1914', '1915', '1916', '1917', '1918', '1919',
       '1920',
       ...
       'gSpikein_ERCC-00158', 'gSpikein_ERCC-00160', 'gSpikein_ERCC-00162',
       'gSpikein_ERCC-00163', 'gSpikein_ERCC-00164', 'gSpikein_ERCC-00165',
       'gSpikein_ERCC-00168', 'gSpikein_ERCC-00170', 'gSpikein_ERCC-00171',
       'gSpikein_phiX174'],
      dtype='object', length=81881)

In [17]:
compute_correlations(dense=dense_10x)

TypeError: '(slice(None, None, None), 0)' is an invalid key